In [ ]:
# PART ONE - Advanced CSV Manipulation - The Power of the DataFrame
# csv.reader and writer are fine for more basic manipulations - but for more advanced, we need pandas!
import pandas as pd # the 'as' just means that we get to call pandas 'pd' in our code
import numpy as np # Numpy is a really nifty library!

df = pd.read_csv("real_loan_data.csv", index_col='id')

# Now we have our DataFrame object! 
# If you need it to be printed in the middle of other code, you must print it out as a raw table by using print()
df.head(5)
# While you can just say df, the .head() method ensures only the first N rows are displayed

In [ ]:
df.tail(5)

In [ ]:
df = df[:-3]
df.tail(5)

In [ ]:
# You can also print out some basic information about your DataFrame

print(df.columns)
print(len(df.columns))
print(df.index)

In [ ]:
# Numpy: Really Cool Number Manipulations!!
# Numpy 1

my_list = np.random.randn(10,4)
print(my_list)

In [ ]:
# Numpy 2: DataFrames Like Numpy!
df_random = pd.DataFrame(np.random.randn(10,4))
df_random.to_csv("my_csv.csv")
print("Done")

In [ ]:
# Numpy 3: arange and reshape
a = np.arange(15).reshape(3, 5)
print(a)
print(a.shape)
print(a.ndim)
b = np.array([6, 7, 8])
# Numpy Array vs Python List - which should I use?
# Basics: Numpy Array are computationally faster and take up less memory, but you lose a lot of Python's cool methods
# and flexibility

In [ ]:
# Numpy 4: Mass-create matrixes of Zeroes and Ones
print(np.zeros( (3,4) ))
print(np.ones( (3,4) ))
print(np.arange( 10, 30, 5 ))
print(np.linspace( 0, 2, 9 ))

In [ ]:
# You can also rename just a single column. But, take special note of inplace below!! An Optional Parameter!
# Inplace=True means that you OVERWRITE the OLD dataframe with the NEW one.
# By default, inplace is false, and that means it creates a DataFrame copy with your change in it
# Some pandas methods don't have an inplace parameter - for those, most make a new dataframe

# Look closely - this will really trip you up if you don't pay attention!
# Without using inplace, you need to assign df to a new variable.
df2 = df.rename(columns={'issue_d': 'issued_date'})

# Using inplace, you can just run it on its own (not setting it equal to anything) and it will overwrite
df.rename(columns={'sub_grade': 'mini_grade'}, inplace=True)
print(df2.head(5))
print(df.head(5))

In [ ]:
# Inserting new columns is also really easy
new_dataframe = pd.DataFrame(columns=['Dummy','B'])
df = pd.concat([df,new_dataframe])
df.head(5)

In [ ]:
# Replace all those NaN with whatever you want.
df = df.fillna(0)

In [ ]:
len(df.columns)

In [ ]:
# To insert/update rows, you can do it either with indexes or with string names (if the rows are named).
df.loc[0] = [2,3,4,5,"1 day",1,1,1,1,1,1,1,1,1] # This will update the first row always
df.loc['Silly'] = [2,3,4,5,"1 day",1,1,1,1,1,1,1,1,1] # This will write a new row since a row named "Silly" doesn't exist
df.loc[len(df)] = [32,3,4,5,"4 days",1,21,1,1,38,87,45,1,1]
# However, note that you need to have the same number of rows in your list as there are rows in the DataFrame
print(df.head(5))
print(df.loc[0])

In [ ]:
# To delete a row or column, there's just a single method
# The key is the axis parameter - 0 means that it's a row, 1 means that it's a column
if 'Dummy' in df.columns: # If there is a Dummy column...
    df.drop('Dummy', axis=1, inplace=True) # Drop it from the columns, since axis is 1
if 'Silly' in df.index: # If there is a row at index 2...
    df.drop('Silly', axis=0, inplace=True) # Drop the row at index 'Silly' from the rows, since axis is 0
print('Dummy' in df.columns)
print('Silly' in df.index)
df.head(5) #AFTER

In [ ]:
# It's also very easy to swap rows and columns
df_new = df.transpose() #They didn't include an inplace parameter for the transpose method. You MUST make a new one
print(df.head(5))
print(df_new.head(5))

In [ ]:
# You can also edit individual cells via the set_value() method
df = df.set_value("54734", 'issue_d', '11-Aug')
# First value is the row index (X), second value is the column name (Y), third value is what to insert
# Unless you're editing a whole row or column in one line, this is the preferred way to edit a single cell
df.head(5)

In [ ]:
df.loc['54345']

In [ ]:
# Fancy Tricks! 
# Trick 1:
# Pandas extends Python's slicing capabilities with their DataFrame objects to allow for STATA/R-like editing
# The below statement does this: 

# For each row in df, check if the loan_status column is NOT equal to "Fully Paid". 
# When it is NOT EQUAL, modify the associated 'term' column to the value 'one million months'
# If it's not, leave it unchanged

df.loc[df['loan_status'] != "Fully Paid", 'term'] = 'one million months'
df.head(5)

In [ ]:
# Trick 2: Print out only certain columns/rows based on conditions

df_new = df.loc[(df["loan_amnt"] > 25000) & (df["loan_status"] != "Fully Paid") & (df["grade"] != "A"), 
                ["member_id","loan_amnt","loan_status","grade"]]
# Note that here I'm not actually modifying any values, although I could. Rather, I'm just displaying a select chunk of
# the spreadsheet
print(df_new.head(5))

df_new = df.loc[((df["loan_amnt"] < 25000) & (df["loan_amnt"] > 30000)) 
                & (df["loan_status"] != "Fully Paid") & (df["grade"] != "A")]
print(df_new.head(5))

# Looking above, you may ask, why are you using & and not 'and'? (You also use | instead of 'or')
# The difference between & and "and" is a pretty complex topic without much meaning to social scientists
# If you're REALLY just that curious, research short-circuiting boolean operators.
# What you DO need to know is that whenever you're inside square brackets [], you need to use &/| intead of and/or.
# The 'and' statement will NOT function properly inside square brackets!

In [ ]:
df['member_id']

In [ ]:
# Trick 3: Applying functions to individual columns (also works for individual rows or all columns or all rows)
def p2f(x):
    if x == 10:
        return x
    else:
        return x-5
#pd.options.display.max_columns = 40
# Note that you can also use lambda functions inside the apply method. However, for more complex operations,
# creating a whole new function is often best
df['int_rate'] = df['int_rate'].apply(p2f)
df['int_rate']

In [ ]:
# Trick 4: Printing out crosstabs
tab = pd.crosstab(df["mini_grade"],df["int_rate"],margins=True)
tab

In [ ]:
# Trick 5: The Power of GroupBy

# First, force the column to be numeric
df['int_rate'] = pd.to_numeric(df['int_rate'],errors='coerce')
# Other than 'coerce, there is also 'raise' and 'ignore'
# 'raise' makes Python give an error if it can't convert a value to a numeric value
# 'ignore' just does nothing to the entire column if it finds a single error, but doesn't alert you

# Now use the power of GroupBy to do amazing things!
my_groups = df['int_rate'].groupby(df['grade'])
print(my_groups.describe())
print(my_groups.mean())

In [ ]:
#Finally, you can spit your DataFrame out as a CSV with one line
df.to_csv("your_spreadsheet.csv")
tab.to_csv("your_crosstab.csv")